In [1]:
import pandas as pd
import numpy as np
import requests
import json

In [2]:
master_table_path = "zones_table/zones_table"                               # Set origin file path

P_NodesDF = pd.read_csv(master_table_path)                                  # Create Data Frame
P_NodesDF = P_NodesDF.drop(labels="Unnamed: 0", axis=1)                     # Drop duplicate number column
P_NodesDF = P_NodesDF.loc[P_NodesDF["ZONA DE CARGA"] != "No Aplica", :]     # Drop rows with unassigned ZONA DE CARGA
P_NodesDF.head(1)

,SISTEMA,CENTRO DE CONTROL REGIONAL,ZONA DE CARGA,CLAVE NODO P,NOMBRE NODO P,NIVEL DE TENSIÓN (kV),TIPO DE CARGA DIRECTAMENTE MODELADA,TIPO DE CARGA INDIRECTAMENTE MODELADA,TIPO DE GENERACION DIRECTAMENTE MODELADA,TIPO DE GENERACION INDIRECTAMENTE MODELADA,ZONA DE OPERACIÓN DE TRANSMISIÓN,GERENCIA REGIONAL DE TRANSMISIÓN,ZONA DE DISTRIBUCIÓN,GERENCIA DIVISIONAL DE DISTRIBUCIÓN,CLAVE DE ENTIDAD FEDERATIVA (INEGI),ENTIDAD FEDERATIVA (INEGI),CLAVE DE MUNICIPIO (INEGI),MUNICIPIO (INEGI),REGION DE TRANSMISION
0,BCA,BAJA CALIFORNIA,ENSENADA,07ACU-115,Acueducto La Mision,115.0,No Aplica,Indirectamente Modelada,No Aplica,No Aplica,COSTA,BAJA CALIFORNIA,ENSENADA,BAJA CALIFORNIA,2,BAJA CALIFORNIA,1,ENSENADA,ENSENADA


In [3]:
# Count unique ZONA DE CARGA for each SISTEMA
Zonas_CargaDF = P_NodesDF.loc[:, ["SISTEMA", "ZONA DE CARGA"]]
Zonas_CargaDF = Zonas_CargaDF.drop_duplicates()
Zonas_CargaDF_Count = Zonas_CargaDF.groupby("SISTEMA").count()
Zonas_CargaDF_Count

,ZONA DE CARGA
SISTEMA,
BCA,4
BCS,3
SIN,101


In [4]:
# Count Unique NODO P for each SISTEMA
NodosDF = P_NodesDF.loc[:, ["SISTEMA", "NOMBRE NODO P"]]
NodosDF = NodosDF.drop_duplicates()
NodosDF_Count = NodosDF.groupby("SISTEMA").count()
NodosDF_Count

,NOMBRE NODO P
SISTEMA,
BCA,101
BCS,28
SIN,2118


In [5]:
# Retrieze names for ZONA and save to lists
BCA_ZonasLS = P_NodesDF.loc[P_NodesDF["SISTEMA"] == "BCA", "ZONA DE CARGA"]
BCA_ZonasLS = BCA_ZonasLS.drop_duplicates()
BCA_ZonasLS = BCA_ZonasLS.tolist()

BCS_ZonasLS = P_NodesDF.loc[P_NodesDF["SISTEMA"] == "BCS", "ZONA DE CARGA"]
BCS_ZonasLS = BCS_ZonasLS.drop_duplicates()
BCS_ZonasLS = BCS_ZonasLS.tolist()

SIN_ZonasLS = P_NodesDF.loc[P_NodesDF["SISTEMA"] == "SIN", "ZONA DE CARGA"]
SIN_ZonasLS = SIN_ZonasLS.drop_duplicates()
SIN_ZonasLS = SIN_ZonasLS.tolist()


In [6]:
# Create lists to build API Requests
year = "2018"
Calendar_LS = []

for month in range(1,13):
    if month in [1, 3, 5, 7, 8, 10, 12]:
        for day in range(1,32):
            if day < 10 and month < 10:
                date = f"{year}/0{str(month)}/0{str(day)}"
                Calendar_LS.append(date)
            elif day < 10 and month >= 10:
                date = f"{year}/{str(month)}/0{str(day)}"
                Calendar_LS.append(date)
            elif day >= 10 and month < 10:
                date = f"{year}/0{str(month)}/{str(day)}"
                Calendar_LS.append(date)
            elif day >= 10 and month >= 10:
                date = f"{year}/{str(month)}/{str(day)}"
                Calendar_LS.append(date)
    elif month in [4, 6, 9, 11]:
        for day in range(1,31):
            if day < 10 and month < 10:
                date = f"{year}/0{str(month)}/0{str(day)}"
                Calendar_LS.append(date)
            elif day < 10 and month >= 10:
                date = f"{year}/{str(month)}/0{str(day)}"
                Calendar_LS.append(date)
            elif day >= 10 and month < 10:
                date = f"{year}/0{str(month)}/{str(day)}"
                Calendar_LS.append(date)
            elif day >= 10 and month >= 10:
                date = f"{year}/{str(month)}/{str(day)}"
                Calendar_LS.append(date)       
    else:
        for day in range(1,29):
            if day < 10 and month < 10:
                date = f"{year}/0{str(month)}/0{str(day)}"
                Calendar_LS.append(date)
            elif day < 10 and month >= 10:
                date = f"{year}/{str(month)}/0{str(day)}"
                Calendar_LS.append(date)
            elif day >= 10 and month < 10:
                date = f"{year}/0{str(month)}/{str(day)}"
                Calendar_LS.append(date)
            elif day >= 10 and month >= 10:
                date = f"{year}/{str(month)}/{str(day)}"
                Calendar_LS.append(date)

# Create dictionary for SISTEMA-ZONAS
Sistemas_dict = {"BCA":BCA_ZonasLS, "BCS":BCS_ZonasLS, "SIN":SIN_ZonasLS}
LS_SIN_ZonasLS = []
Zonas_aux = []
flag_sist = 0
while flag_sist < len(Sistemas_dict["SIN"]):
    if flag_sist in [9, 19, 29, 39, 49, 59, 69, 79, 89, 99]:
        zn = Sistemas_dict["SIN"][flag_sist]
        Zonas_aux.append(zn)
        LS_SIN_ZonasLS.append(Zonas_aux)
        Zonas_aux = []
        flag_sist += 1
    else:
        zn = Sistemas_dict["SIN"][flag_sist]
        Zonas_aux.append(zn)
        flag_sist += 1
    if flag_sist == 100:
        Zonas_aux = []   
        zn = Sistemas_dict["SIN"][flag_sist]
        Zonas_aux.append(zn)
        LS_SIN_ZonasLS.append(Zonas_aux)
        break
Sistemas_dict["SIN"] = LS_SIN_ZonasLS

In [7]:
#Create Lists for API Requests - MDA               
Sistemas_MDA = []
Zonas_MDA = []
Fechas_MDA = []
Horas_MDA = []
pz_MDA = []
pz_ene_MDA = []
pz_per_MDA = []
pz_cng_MDA = []

In [8]:
# Make API requests - MDA
#Sistemas BC y BCS 

TEST = 0

for SISTEMA in Sistemas_dict:
    
    if SISTEMA != "SIN":      
        
        ZONAS= Sistemas_dict[SISTEMA]
        ZTEXT = ','.join(ZONAS)
           
        n = 0
        
        while n < 365:
            if n == 364:
                DUNO = Calendar_LS[n]
                DDOS = Calendar_LS[n]
                THoras = 24
            else:
                
                DUNO = Calendar_LS[n]
                DDOS = Calendar_LS[n+6]
                THoras = 168
            
            url_new = f"https://ws01.cenace.gob.mx:8082/SWPEND/SIM/{SISTEMA}/MDA/{ZTEXT}/{DUNO}/{DDOS}/JSON"
            url_new= url_new.replace(" ","-")
            response = requests.get(url_new)
            response_JSON = response.json()
            
            if response_JSON["status"] == "OK":
                
                for Z in range(len(ZONAS)):
                    
                    for HORA in range(THoras):
                        
                        Sistemas_MDA.append(response_JSON["sistema"])
                        Zonas_MDA.append(response_JSON["Resultados"][Z]["zona_carga"])
                        try:
                            Fechas_MDA.append(response_JSON["Resultados"][Z]["Valores"][HORA]["fecha"])
                        except: 
                            Fechas_MDA.append("NULL")
                        try: 
                            Horas_MDA.append(response_JSON["Resultados"][Z]["Valores"][HORA]["hora"])
                        except: 
                            Horas_MDA.append("NULL")
                        try:
                            pz_MDA.append(response_JSON["Resultados"][Z]["Valores"][HORA]["pz"])
                        except:
                            pz_MDA.append("NULL")
                        try:
                            pz_ene_MDA.append(response_JSON["Resultados"][Z]["Valores"][HORA]["pz_ene"])
                        except:
                            pz_ene_MDA.append("NULL")
                        try:
                            pz_per_MDA.append(response_JSON["Resultados"][Z]["Valores"][HORA]["pz_per"])
                        except:
                            pz_per_MDA.append("NULL")
                        try:
                            pz_cng_MDA.append(response_JSON["Resultados"][Z]["Valores"][HORA]["pz_cng"])
                        except:
                            pz_cng_MDA.append("NULL")
            n= n + 7
            TEST= TEST+1
            print(f"Processing: MDA | {SISTEMA} |{ZONAS} | TEST:{TEST}  ")

    

Processing: MDA | BCA |['ENSENADA', 'MEXICALI', 'SANLUIS', 'TIJUANA'] | TEST:1  
Processing: MDA | BCA |['ENSENADA', 'MEXICALI', 'SANLUIS', 'TIJUANA'] | TEST:2  
Processing: MDA | BCA |['ENSENADA', 'MEXICALI', 'SANLUIS', 'TIJUANA'] | TEST:3  
Processing: MDA | BCA |['ENSENADA', 'MEXICALI', 'SANLUIS', 'TIJUANA'] | TEST:4  
Processing: MDA | BCA |['ENSENADA', 'MEXICALI', 'SANLUIS', 'TIJUANA'] | TEST:5  
Processing: MDA | BCA |['ENSENADA', 'MEXICALI', 'SANLUIS', 'TIJUANA'] | TEST:6  
Processing: MDA | BCA |['ENSENADA', 'MEXICALI', 'SANLUIS', 'TIJUANA'] | TEST:7  
Processing: MDA | BCA |['ENSENADA', 'MEXICALI', 'SANLUIS', 'TIJUANA'] | TEST:8  
Processing: MDA | BCA |['ENSENADA', 'MEXICALI', 'SANLUIS', 'TIJUANA'] | TEST:9  
Processing: MDA | BCA |['ENSENADA', 'MEXICALI', 'SANLUIS', 'TIJUANA'] | TEST:10  
Processing: MDA | BCA |['ENSENADA', 'MEXICALI', 'SANLUIS', 'TIJUANA'] | TEST:11  
Processing: MDA | BCA |['ENSENADA', 'MEXICALI', 'SANLUIS', 'TIJUANA'] | TEST:12  
Processing: MDA | BCA |['

Processing: MDA | BCS |['CONSTITUCION', 'LA PAZ', 'LOS CABOS'] | TEST:106  


In [9]:
#Make API Requests - MDA
#Sistema Interconectado Nacional (SIN) 

TEST = 0

SISTEMA = "SIN"
    
for GRUPO in range(len(Sistemas_dict[SISTEMA])):      
        
    ZONAS= Sistemas_dict["SIN"][GRUPO]
    ZTEXT = ','.join(ZONAS)
           
    n = 0
        
    while n < 365:
        if n == 364:
            DUNO = Calendar_LS[n]
            DDOS = Calendar_LS[n]
            THoras = 24
        else:
                
            DUNO = Calendar_LS[n]
            DDOS = Calendar_LS[n+6]
            THoras = 168
            
        url_new = f"https://ws01.cenace.gob.mx:8082/SWPEND/SIM/{SISTEMA}/MDA/{ZTEXT}/{DUNO}/{DDOS}/JSON"
        url_new= url_new.replace(" ","-")
        response = requests.get(url_new)
        response_JSON = response.json()
            
        if response_JSON["status"] == "OK":
                
            for Z in range(len(ZONAS)):
                    
                for HORA in range(THoras):
                        
                    Sistemas_MDA.append(response_JSON["sistema"])
                    Zonas_MDA.append(response_JSON["Resultados"][Z]["zona_carga"])
                    try:
                        Fechas_MDA.append(response_JSON["Resultados"][Z]["Valores"][HORA]["fecha"])
                    except: 
                        Fechas_MDA.append("NULL")
                    try: 
                        Horas_MDA.append(response_JSON["Resultados"][Z]["Valores"][HORA]["hora"])
                    except: 
                        Horas_MDA.append("NULL")
                    try:
                        pz_MDA.append(response_JSON["Resultados"][Z]["Valores"][HORA]["pz"])
                    except:
                        pz_MDA.append("NULL")
                    try:
                        pz_ene_MDA.append(response_JSON["Resultados"][Z]["Valores"][HORA]["pz_ene"])
                    except:
                        pz_ene_MDA.append("NULL")
                    try:
                        pz_per_MDA.append(response_JSON["Resultados"][Z]["Valores"][HORA]["pz_per"])
                    except:
                        pz_per_MDA.append("NULL")
                    try:
                        pz_cng_MDA.append(response_JSON["Resultados"][Z]["Valores"][HORA]["pz_cng"])
                    except:
                        pz_cng_MDA.append("NULL")
        n= n + 7
        TEST= TEST+1
        print(f"Processing: MDA | {SISTEMA} |{ZONAS} | TEST:{TEST} ")


Processing: MDA | SIN |['CENTRO ORIENTE', 'CENTRO SUR', 'LAZARO CARDENAS', 'VDM CENTRO', 'VDM NORTE', 'VDM SUR', 'HUASTECA', 'HUEJUTLA', 'MATAMOROS', 'MONCLOVA'] | TEST:1 
Processing: MDA | SIN |['CENTRO ORIENTE', 'CENTRO SUR', 'LAZARO CARDENAS', 'VDM CENTRO', 'VDM NORTE', 'VDM SUR', 'HUASTECA', 'HUEJUTLA', 'MATAMOROS', 'MONCLOVA'] | TEST:2 
Processing: MDA | SIN |['CENTRO ORIENTE', 'CENTRO SUR', 'LAZARO CARDENAS', 'VDM CENTRO', 'VDM NORTE', 'VDM SUR', 'HUASTECA', 'HUEJUTLA', 'MATAMOROS', 'MONCLOVA'] | TEST:3 
Processing: MDA | SIN |['CENTRO ORIENTE', 'CENTRO SUR', 'LAZARO CARDENAS', 'VDM CENTRO', 'VDM NORTE', 'VDM SUR', 'HUASTECA', 'HUEJUTLA', 'MATAMOROS', 'MONCLOVA'] | TEST:4 
Processing: MDA | SIN |['CENTRO ORIENTE', 'CENTRO SUR', 'LAZARO CARDENAS', 'VDM CENTRO', 'VDM NORTE', 'VDM SUR', 'HUASTECA', 'HUEJUTLA', 'MATAMOROS', 'MONCLOVA'] | TEST:5 
Processing: MDA | SIN |['CENTRO ORIENTE', 'CENTRO SUR', 'LAZARO CARDENAS', 'VDM CENTRO', 'VDM NORTE', 'VDM SUR', 'HUASTECA', 'HUEJUTLA', 'MA

Processing: MDA | SIN |['CENTRO ORIENTE', 'CENTRO SUR', 'LAZARO CARDENAS', 'VDM CENTRO', 'VDM NORTE', 'VDM SUR', 'HUASTECA', 'HUEJUTLA', 'MATAMOROS', 'MONCLOVA'] | TEST:49 
Processing: MDA | SIN |['CENTRO ORIENTE', 'CENTRO SUR', 'LAZARO CARDENAS', 'VDM CENTRO', 'VDM NORTE', 'VDM SUR', 'HUASTECA', 'HUEJUTLA', 'MATAMOROS', 'MONCLOVA'] | TEST:50 
Processing: MDA | SIN |['CENTRO ORIENTE', 'CENTRO SUR', 'LAZARO CARDENAS', 'VDM CENTRO', 'VDM NORTE', 'VDM SUR', 'HUASTECA', 'HUEJUTLA', 'MATAMOROS', 'MONCLOVA'] | TEST:51 
Processing: MDA | SIN |['CENTRO ORIENTE', 'CENTRO SUR', 'LAZARO CARDENAS', 'VDM CENTRO', 'VDM NORTE', 'VDM SUR', 'HUASTECA', 'HUEJUTLA', 'MATAMOROS', 'MONCLOVA'] | TEST:52 
Processing: MDA | SIN |['CENTRO ORIENTE', 'CENTRO SUR', 'LAZARO CARDENAS', 'VDM CENTRO', 'VDM NORTE', 'VDM SUR', 'HUASTECA', 'HUEJUTLA', 'MATAMOROS', 'MONCLOVA'] | TEST:53 
Processing: MDA | SIN |['MONTEMORELOS', 'MONTERREY', 'NUEVO LAREDO', 'PIEDRAS NEGRAS', 'REYNOSA', 'SABINAS', 'SALTILLO', 'TAMPICO', 'VI

Processing: MDA | SIN |['MONTEMORELOS', 'MONTERREY', 'NUEVO LAREDO', 'PIEDRAS NEGRAS', 'REYNOSA', 'SABINAS', 'SALTILLO', 'TAMPICO', 'VICTORIA', 'CABORCA'] | TEST:99 
Processing: MDA | SIN |['MONTEMORELOS', 'MONTERREY', 'NUEVO LAREDO', 'PIEDRAS NEGRAS', 'REYNOSA', 'SABINAS', 'SALTILLO', 'TAMPICO', 'VICTORIA', 'CABORCA'] | TEST:100 
Processing: MDA | SIN |['MONTEMORELOS', 'MONTERREY', 'NUEVO LAREDO', 'PIEDRAS NEGRAS', 'REYNOSA', 'SABINAS', 'SALTILLO', 'TAMPICO', 'VICTORIA', 'CABORCA'] | TEST:101 
Processing: MDA | SIN |['MONTEMORELOS', 'MONTERREY', 'NUEVO LAREDO', 'PIEDRAS NEGRAS', 'REYNOSA', 'SABINAS', 'SALTILLO', 'TAMPICO', 'VICTORIA', 'CABORCA'] | TEST:102 
Processing: MDA | SIN |['MONTEMORELOS', 'MONTERREY', 'NUEVO LAREDO', 'PIEDRAS NEGRAS', 'REYNOSA', 'SABINAS', 'SALTILLO', 'TAMPICO', 'VICTORIA', 'CABORCA'] | TEST:103 
Processing: MDA | SIN |['MONTEMORELOS', 'MONTERREY', 'NUEVO LAREDO', 'PIEDRAS NEGRAS', 'REYNOSA', 'SABINAS', 'SALTILLO', 'TAMPICO', 'VICTORIA', 'CABORCA'] | TEST:104 

Processing: MDA | SIN |['CULIACAN', 'GUASAVE', 'GUAYMAS', 'HERMOSILLO', 'LOS MOCHIS', 'MAZATLAN', 'NAVOJOA', 'NOGALES', 'OBREGON', 'CAMARGO'] | TEST:152 
Processing: MDA | SIN |['CULIACAN', 'GUASAVE', 'GUAYMAS', 'HERMOSILLO', 'LOS MOCHIS', 'MAZATLAN', 'NAVOJOA', 'NOGALES', 'OBREGON', 'CAMARGO'] | TEST:153 
Processing: MDA | SIN |['CULIACAN', 'GUASAVE', 'GUAYMAS', 'HERMOSILLO', 'LOS MOCHIS', 'MAZATLAN', 'NAVOJOA', 'NOGALES', 'OBREGON', 'CAMARGO'] | TEST:154 
Processing: MDA | SIN |['CULIACAN', 'GUASAVE', 'GUAYMAS', 'HERMOSILLO', 'LOS MOCHIS', 'MAZATLAN', 'NAVOJOA', 'NOGALES', 'OBREGON', 'CAMARGO'] | TEST:155 
Processing: MDA | SIN |['CULIACAN', 'GUASAVE', 'GUAYMAS', 'HERMOSILLO', 'LOS MOCHIS', 'MAZATLAN', 'NAVOJOA', 'NOGALES', 'OBREGON', 'CAMARGO'] | TEST:156 
Processing: MDA | SIN |['CULIACAN', 'GUASAVE', 'GUAYMAS', 'HERMOSILLO', 'LOS MOCHIS', 'MAZATLAN', 'NAVOJOA', 'NOGALES', 'OBREGON', 'CAMARGO'] | TEST:157 
Processing: MDA | SIN |['CULIACAN', 'GUASAVE', 'GUAYMAS', 'HERMOSILLO', 'LOS

Processing: MDA | SIN |['CASAS GRANDES', 'CHIHUAHUA', 'CUAUHTEMOC', 'DURANGO', 'JUAREZ', 'LAGUNA', 'AGUASCALIENTES', 'APATZINGAN', 'CELAYA', 'CIENEGA'] | TEST:203 
Processing: MDA | SIN |['CASAS GRANDES', 'CHIHUAHUA', 'CUAUHTEMOC', 'DURANGO', 'JUAREZ', 'LAGUNA', 'AGUASCALIENTES', 'APATZINGAN', 'CELAYA', 'CIENEGA'] | TEST:204 
Processing: MDA | SIN |['CASAS GRANDES', 'CHIHUAHUA', 'CUAUHTEMOC', 'DURANGO', 'JUAREZ', 'LAGUNA', 'AGUASCALIENTES', 'APATZINGAN', 'CELAYA', 'CIENEGA'] | TEST:205 
Processing: MDA | SIN |['CASAS GRANDES', 'CHIHUAHUA', 'CUAUHTEMOC', 'DURANGO', 'JUAREZ', 'LAGUNA', 'AGUASCALIENTES', 'APATZINGAN', 'CELAYA', 'CIENEGA'] | TEST:206 
Processing: MDA | SIN |['CASAS GRANDES', 'CHIHUAHUA', 'CUAUHTEMOC', 'DURANGO', 'JUAREZ', 'LAGUNA', 'AGUASCALIENTES', 'APATZINGAN', 'CELAYA', 'CIENEGA'] | TEST:207 
Processing: MDA | SIN |['CASAS GRANDES', 'CHIHUAHUA', 'CUAUHTEMOC', 'DURANGO', 'JUAREZ', 'LAGUNA', 'AGUASCALIENTES', 'APATZINGAN', 'CELAYA', 'CIENEGA'] | TEST:208 
Processing: MDA 

...

Processing: MDA | SIN |['MINAS', 'MORELIA', 'QUERETARO', 'SALVATIERRA', 'SAN JUAN DEL RIO', 'SAN LUIS POTOSI', 'TEPIC VALLARTA', 'URUAPAN', 'ZACAPU', 'ZACATECAS'] | TEST:302 
Processing: MDA | SIN |['MINAS', 'MORELIA', 'QUERETARO', 'SALVATIERRA', 'SAN JUAN DEL RIO', 'SAN LUIS POTOSI', 'TEPIC VALLARTA', 'URUAPAN', 'ZACAPU', 'ZACATECAS'] | TEST:303 
Processing: MDA | SIN |['MINAS', 'MORELIA', 'QUERETARO', 'SALVATIERRA', 'SAN JUAN DEL RIO', 'SAN LUIS POTOSI', 'TEPIC VALLARTA', 'URUAPAN', 'ZACAPU', 'ZACATECAS'] | TEST:304 
Processing: MDA | SIN |['MINAS', 'MORELIA', 'QUERETARO', 'SALVATIERRA', 'SAN JUAN DEL RIO', 'SAN LUIS POTOSI', 'TEPIC VALLARTA', 'URUAPAN', 'ZACAPU', 'ZACATECAS'] | TEST:305 
Processing: MDA | SIN |['MINAS', 'MORELIA', 'QUERETARO', 'SALVATIERRA', 'SAN JUAN DEL RIO', 'SAN LUIS POTOSI', 'TEPIC VALLARTA', 'URUAPAN', 'ZACAPU', 'ZACATECAS'] | TEST:306 
Processing: MDA | SIN |['MINAS', 'MORELIA', 'QUERETARO', 'SALVATIERRA', 'SAN JUAN DEL RIO', 'SAN LUIS POTOSI', 'TEPIC VALLART

Processing: MDA | SIN |['ZAMORA', 'ZAPOTLAN', 'ACAPULCO', 'CHILPANCINGO', 'CHONTALPA', 'COATZACOALCOS', 'CORDOBA', 'CUAUTLA', 'CUERNAVACA', 'HUAJUAPAN'] | TEST:351 
Processing: MDA | SIN |['ZAMORA', 'ZAPOTLAN', 'ACAPULCO', 'CHILPANCINGO', 'CHONTALPA', 'COATZACOALCOS', 'CORDOBA', 'CUAUTLA', 'CUERNAVACA', 'HUAJUAPAN'] | TEST:352 
Processing: MDA | SIN |['ZAMORA', 'ZAPOTLAN', 'ACAPULCO', 'CHILPANCINGO', 'CHONTALPA', 'COATZACOALCOS', 'CORDOBA', 'CUAUTLA', 'CUERNAVACA', 'HUAJUAPAN'] | TEST:353 
Processing: MDA | SIN |['ZAMORA', 'ZAPOTLAN', 'ACAPULCO', 'CHILPANCINGO', 'CHONTALPA', 'COATZACOALCOS', 'CORDOBA', 'CUAUTLA', 'CUERNAVACA', 'HUAJUAPAN'] | TEST:354 
Processing: MDA | SIN |['ZAMORA', 'ZAPOTLAN', 'ACAPULCO', 'CHILPANCINGO', 'CHONTALPA', 'COATZACOALCOS', 'CORDOBA', 'CUAUTLA', 'CUERNAVACA', 'HUAJUAPAN'] | TEST:355 
Processing: MDA | SIN |['ZAMORA', 'ZAPOTLAN', 'ACAPULCO', 'CHILPANCINGO', 'CHONTALPA', 'COATZACOALCOS', 'CORDOBA', 'CUAUTLA', 'CUERNAVACA', 'HUAJUAPAN'] | TEST:356 
Processing

Processing: MDA | SIN |['HUATULCO', 'IGUALA', 'IZUCAR', 'LOS RIOS', 'LOS TUXTLAS', 'MORELOS', 'OAXACA', 'ORIZABA', 'POZA RICA', 'PUEBLA'] | TEST:404 
Processing: MDA | SIN |['HUATULCO', 'IGUALA', 'IZUCAR', 'LOS RIOS', 'LOS TUXTLAS', 'MORELOS', 'OAXACA', 'ORIZABA', 'POZA RICA', 'PUEBLA'] | TEST:405 
Processing: MDA | SIN |['HUATULCO', 'IGUALA', 'IZUCAR', 'LOS RIOS', 'LOS TUXTLAS', 'MORELOS', 'OAXACA', 'ORIZABA', 'POZA RICA', 'PUEBLA'] | TEST:406 
Processing: MDA | SIN |['HUATULCO', 'IGUALA', 'IZUCAR', 'LOS RIOS', 'LOS TUXTLAS', 'MORELOS', 'OAXACA', 'ORIZABA', 'POZA RICA', 'PUEBLA'] | TEST:407 
Processing: MDA | SIN |['HUATULCO', 'IGUALA', 'IZUCAR', 'LOS RIOS', 'LOS TUXTLAS', 'MORELOS', 'OAXACA', 'ORIZABA', 'POZA RICA', 'PUEBLA'] | TEST:408 
Processing: MDA | SIN |['HUATULCO', 'IGUALA', 'IZUCAR', 'LOS RIOS', 'LOS TUXTLAS', 'MORELOS', 'OAXACA', 'ORIZABA', 'POZA RICA', 'PUEBLA'] | TEST:409 
Processing: MDA | SIN |['HUATULCO', 'IGUALA', 'IZUCAR', 'LOS RIOS', 'LOS TUXTLAS', 'MORELOS', 'OAXAC

Processing: MDA | SIN |['SAN CRISTOBAL', 'SAN MARTIN', 'TAPACHULA', 'TECAMACHALCO', 'TEHUACAN', 'TEHUANTEPEC', 'TEZIUTLAN', 'TLAXCALA', 'TUXTLA', 'VERACRUZ'] | TEST:455 
Processing: MDA | SIN |['SAN CRISTOBAL', 'SAN MARTIN', 'TAPACHULA', 'TECAMACHALCO', 'TEHUACAN', 'TEHUANTEPEC', 'TEZIUTLAN', 'TLAXCALA', 'TUXTLA', 'VERACRUZ'] | TEST:456 
Processing: MDA | SIN |['SAN CRISTOBAL', 'SAN MARTIN', 'TAPACHULA', 'TECAMACHALCO', 'TEHUACAN', 'TEHUANTEPEC', 'TEZIUTLAN', 'TLAXCALA', 'TUXTLA', 'VERACRUZ'] | TEST:457 
Processing: MDA | SIN |['SAN CRISTOBAL', 'SAN MARTIN', 'TAPACHULA', 'TECAMACHALCO', 'TEHUACAN', 'TEHUANTEPEC', 'TEZIUTLAN', 'TLAXCALA', 'TUXTLA', 'VERACRUZ'] | TEST:458 
Processing: MDA | SIN |['SAN CRISTOBAL', 'SAN MARTIN', 'TAPACHULA', 'TECAMACHALCO', 'TEHUACAN', 'TEHUANTEPEC', 'TEZIUTLAN', 'TLAXCALA', 'TUXTLA', 'VERACRUZ'] | TEST:459 
Processing: MDA | SIN |['SAN CRISTOBAL', 'SAN MARTIN', 'TAPACHULA', 'TECAMACHALCO', 'TEHUACAN', 'TEHUANTEPEC', 'TEZIUTLAN', 'TLAXCALA', 'TUXTLA', 'VER

Processing: MDA | SIN |['VILLAHERMOSA', 'XALAPA', 'ZIHUATANEJO', 'CAMPECHE', 'CANCUN', 'CARMEN', 'CHETUMAL', 'MERIDA', 'MOTUL TIZIMIN', 'RIVIERA MAYA'] | TEST:505 
Processing: MDA | SIN |['VILLAHERMOSA', 'XALAPA', 'ZIHUATANEJO', 'CAMPECHE', 'CANCUN', 'CARMEN', 'CHETUMAL', 'MERIDA', 'MOTUL TIZIMIN', 'RIVIERA MAYA'] | TEST:506 
Processing: MDA | SIN |['VILLAHERMOSA', 'XALAPA', 'ZIHUATANEJO', 'CAMPECHE', 'CANCUN', 'CARMEN', 'CHETUMAL', 'MERIDA', 'MOTUL TIZIMIN', 'RIVIERA MAYA'] | TEST:507 
Processing: MDA | SIN |['VILLAHERMOSA', 'XALAPA', 'ZIHUATANEJO', 'CAMPECHE', 'CANCUN', 'CARMEN', 'CHETUMAL', 'MERIDA', 'MOTUL TIZIMIN', 'RIVIERA MAYA'] | TEST:508 
Processing: MDA | SIN |['VILLAHERMOSA', 'XALAPA', 'ZIHUATANEJO', 'CAMPECHE', 'CANCUN', 'CARMEN', 'CHETUMAL', 'MERIDA', 'MOTUL TIZIMIN', 'RIVIERA MAYA'] | TEST:509 
Processing: MDA | SIN |['VILLAHERMOSA', 'XALAPA', 'ZIHUATANEJO', 'CAMPECHE', 'CANCUN', 'CARMEN', 'CHETUMAL', 'MERIDA', 'MOTUL TIZIMIN', 'RIVIERA MAYA'] | TEST:510 
Processing: MDA 

In [10]:
#Dataframe for MDA prices

MT_DF_MDA = pd.DataFrame({
    "SISTEMA":Sistemas_MDA,
    "ZONA":Zonas_MDA,
    "FECHA":Fechas_MDA,
    "HORA":Horas_MDA,
    "PRECIO MDA":pz_MDA,
    "PRECIO ENERGIA MDA":pz_ene_MDA,
    "PRECIO PERDIDA MDA":pz_per_MDA,
    "PRECIO CONGESTION MDA":pz_cng_MDA,
})
MT_DF_MDA= MT_DF_MDA.sort_values(["SISTEMA","ZONA","FECHA"], ascending=[True, True,True])
MT_DF_MDA

,SISTEMA,ZONA,FECHA,HORA,PRECIO MDA,PRECIO ENERGIA MDA,PRECIO PERDIDA MDA,PRECIO CONGESTION MDA
0,BCA,ENSENADA,2018-01-01,1,192.06,187.15,4.9,0
1,BCA,ENSENADA,2018-01-01,2,818.25,799.03,19.22,0
2,BCA,ENSENADA,2018-01-01,3,799,780.7,18.3,0
3,BCA,ENSENADA,2018-01-01,4,829.04,810.53,18.51,0
4,BCA,ENSENADA,2018-01-01,5,508.93,497.33,11.61,0
5,BCA,ENSENADA,2018-01-01,6,498.35,486.89,11.46,0
6,BCA,ENSENADA,2018-01-01,7,247.89,242.4,5.49,0
7,BCA,ENSENADA,2018-01-01,8,241.47,236.47,5,0
8,BCA,ENSENADA,2018-01-01,9,-56.82,-55.65,-1.17,0
9,BCA,ENSENADA,2018-01-01,10,515.38,504.36,11.03,0


In [11]:
MT_DF_MDA.to_csv("Master_Table_MDA.csv")

In [12]:
#Create Lists for API Requests - MTR               
Sistemas_MTR = []
Zonas_MTR = []
Fechas_MTR = []
Horas_MTR = []
pz_MTR = []
pz_ene_MTR = []
pz_per_MTR = []
pz_cng_MTR = []

In [13]:
# Make API requests - MTR
#Sistemas BC y BCS 

TEST = 0

for SISTEMA in Sistemas_dict:
    
    if SISTEMA != "SIN":      
        
        ZONAS= Sistemas_dict[SISTEMA]
        ZTEXT = ','.join(ZONAS)
           
        n = 0
        
        while n < 365:
            if n == 364:
                DUNO = Calendar_LS[n]
                DDOS = Calendar_LS[n]
                THoras = 24
            else:
                
                DUNO = Calendar_LS[n]
                DDOS = Calendar_LS[n+6]
                THoras = 168
            
            url_new = f"https://ws01.cenace.gob.mx:8082/SWPEND/SIM/{SISTEMA}/MTR/{ZTEXT}/{DUNO}/{DDOS}/JSON"
            url_new= url_new.replace(" ","-")
            response = requests.get(url_new)
            response_JSON = response.json()
            
            if response_JSON["status"] == "OK":
                
                for Z in range(len(ZONAS)):
                    
                    for HORA in range(THoras):
                        
                        Sistemas_MTR.append(response_JSON["sistema"])
                        Zonas_MTR.append(response_JSON["Resultados"][Z]["zona_carga"])
                        try:
                            Fechas_MTR.append(response_JSON["Resultados"][Z]["Valores"][HORA]["fecha"])
                        except: 
                            Fechas_MTR.append("NULL")
                        try: 
                            Horas_MTR.append(response_JSON["Resultados"][Z]["Valores"][HORA]["hora"])
                        except: 
                            Horas_MTR.append("NULL")
                        try:
                            pz_MTR.append(response_JSON["Resultados"][Z]["Valores"][HORA]["pz"])
                        except:
                            pz_MTR.append("NULL")
                        try:
                            pz_ene_MTR.append(response_JSON["Resultados"][Z]["Valores"][HORA]["pz_ene"])
                        except:
                            pz_ene_MTR.append("NULL")
                        try:
                            pz_per_MTR.append(response_JSON["Resultados"][Z]["Valores"][HORA]["pz_per"])
                        except:
                            pz_per_MTR.append("NULL")
                        try:
                            pz_cng_MTR.append(response_JSON["Resultados"][Z]["Valores"][HORA]["pz_cng"])
                        except:
                            pz_cng_MTR.append("NULL")
            n= n + 7
            TEST= TEST+1
            print(f"Processing: MTR | {SISTEMA} |{ZONAS} | TEST:{TEST} ")


Processing: MTR | BCA |['ENSENADA', 'MEXICALI', 'SANLUIS', 'TIJUANA'] | TEST:1 
Processing: MTR | BCA |['ENSENADA', 'MEXICALI', 'SANLUIS', 'TIJUANA'] | TEST:2 
Processing: MTR | BCA |['ENSENADA', 'MEXICALI', 'SANLUIS', 'TIJUANA'] | TEST:3 
Processing: MTR | BCA |['ENSENADA', 'MEXICALI', 'SANLUIS', 'TIJUANA'] | TEST:4 
Processing: MTR | BCA |['ENSENADA', 'MEXICALI', 'SANLUIS', 'TIJUANA'] | TEST:5 
Processing: MTR | BCA |['ENSENADA', 'MEXICALI', 'SANLUIS', 'TIJUANA'] | TEST:6 
Processing: MTR | BCA |['ENSENADA', 'MEXICALI', 'SANLUIS', 'TIJUANA'] | TEST:7 
Processing: MTR | BCA |['ENSENADA', 'MEXICALI', 'SANLUIS', 'TIJUANA'] | TEST:8 
Processing: MTR | BCA |['ENSENADA', 'MEXICALI', 'SANLUIS', 'TIJUANA'] | TEST:9 
Processing: MTR | BCA |['ENSENADA', 'MEXICALI', 'SANLUIS', 'TIJUANA'] | TEST:10 
Processing: MTR | BCA |['ENSENADA', 'MEXICALI', 'SANLUIS', 'TIJUANA'] | TEST:11 
Processing: MTR | BCA |['ENSENADA', 'MEXICALI', 'SANLUIS', 'TIJUANA'] | TEST:12 
Processing: MTR | BCA |['ENSENADA', '

In [14]:
#Make API Requests - MTR
#Sistema Interconectado Nacional (SIN) 

TEST = 0

SISTEMA = "SIN"
    
for GRUPO in range(len(Sistemas_dict[SISTEMA])):      
        
    ZONAS= Sistemas_dict["SIN"][GRUPO]
    ZTEXT = ','.join(ZONAS)
           
    n = 0
        
    while n < 365:
        if n == 364:
            DUNO = Calendar_LS[n]
            DDOS = Calendar_LS[n]
            THoras = 24
        else:
                
            DUNO = Calendar_LS[n]
            DDOS = Calendar_LS[n+6]
            THoras = 168
            
        url_new = f"https://ws01.cenace.gob.mx:8082/SWPEND/SIM/{SISTEMA}/MTR/{ZTEXT}/{DUNO}/{DDOS}/JSON"
        url_new= url_new.replace(" ","-")
        response = requests.get(url_new)
        response_JSON = response.json()
            
        if response_JSON["status"] == "OK":
                
            for Z in range(len(ZONAS)):
                    
                for HORA in range(THoras):
                        
                    Sistemas_MTR.append(response_JSON["sistema"])
                    Zonas_MTR.append(response_JSON["Resultados"][Z]["zona_carga"])
                    try:
                        Fechas_MTR.append(response_JSON["Resultados"][Z]["Valores"][HORA]["fecha"])
                    except: 
                        Fechas_MTR.append("NULL")
                    try: 
                        Horas_MTR.append(response_JSON["Resultados"][Z]["Valores"][HORA]["hora"])
                    except: 
                        Horas_MTR.append("NULL")
                    try:
                        pz_MTR.append(response_JSON["Resultados"][Z]["Valores"][HORA]["pz"])
                    except:
                        pz_MTR.append("NULL")
                    try:
                        pz_ene_MTR.append(response_JSON["Resultados"][Z]["Valores"][HORA]["pz_ene"])
                    except:
                        pz_ene_MTR.append("NULL")
                    try:
                        pz_per_MTR.append(response_JSON["Resultados"][Z]["Valores"][HORA]["pz_per"])
                    except:
                        pz_per_MTR.append("NULL")
                    try:
                        pz_cng_MTR.append(response_JSON["Resultados"][Z]["Valores"][HORA]["pz_cng"])
                    except:
                        pz_cng_MTR.append("NULL")
        n= n + 7
        TEST= TEST+1
        print(f"Processing: {SISTEMA} |{ZONAS} | TEST:{TEST} ")


Processing: SIN |['CENTRO ORIENTE', 'CENTRO SUR', 'LAZARO CARDENAS', 'VDM CENTRO', 'VDM NORTE', 'VDM SUR', 'HUASTECA', 'HUEJUTLA', 'MATAMOROS', 'MONCLOVA'] | TEST:1 
Processing: SIN |['CENTRO ORIENTE', 'CENTRO SUR', 'LAZARO CARDENAS', 'VDM CENTRO', 'VDM NORTE', 'VDM SUR', 'HUASTECA', 'HUEJUTLA', 'MATAMOROS', 'MONCLOVA'] | TEST:2 
Processing: SIN |['CENTRO ORIENTE', 'CENTRO SUR', 'LAZARO CARDENAS', 'VDM CENTRO', 'VDM NORTE', 'VDM SUR', 'HUASTECA', 'HUEJUTLA', 'MATAMOROS', 'MONCLOVA'] | TEST:3 
Processing: SIN |['CENTRO ORIENTE', 'CENTRO SUR', 'LAZARO CARDENAS', 'VDM CENTRO', 'VDM NORTE', 'VDM SUR', 'HUASTECA', 'HUEJUTLA', 'MATAMOROS', 'MONCLOVA'] | TEST:4 
Processing: SIN |['CENTRO ORIENTE', 'CENTRO SUR', 'LAZARO CARDENAS', 'VDM CENTRO', 'VDM NORTE', 'VDM SUR', 'HUASTECA', 'HUEJUTLA', 'MATAMOROS', 'MONCLOVA'] | TEST:5 
Processing: SIN |['CENTRO ORIENTE', 'CENTRO SUR', 'LAZARO CARDENAS', 'VDM CENTRO', 'VDM NORTE', 'VDM SUR', 'HUASTECA', 'HUEJUTLA', 'MATAMOROS', 'MONCLOVA'] | TEST:6 
Proc

Processing: SIN |['CENTRO ORIENTE', 'CENTRO SUR', 'LAZARO CARDENAS', 'VDM CENTRO', 'VDM NORTE', 'VDM SUR', 'HUASTECA', 'HUEJUTLA', 'MATAMOROS', 'MONCLOVA'] | TEST:51 
Processing: SIN |['CENTRO ORIENTE', 'CENTRO SUR', 'LAZARO CARDENAS', 'VDM CENTRO', 'VDM NORTE', 'VDM SUR', 'HUASTECA', 'HUEJUTLA', 'MATAMOROS', 'MONCLOVA'] | TEST:52 
Processing: SIN |['CENTRO ORIENTE', 'CENTRO SUR', 'LAZARO CARDENAS', 'VDM CENTRO', 'VDM NORTE', 'VDM SUR', 'HUASTECA', 'HUEJUTLA', 'MATAMOROS', 'MONCLOVA'] | TEST:53 
Processing: SIN |['MONTEMORELOS', 'MONTERREY', 'NUEVO LAREDO', 'PIEDRAS NEGRAS', 'REYNOSA', 'SABINAS', 'SALTILLO', 'TAMPICO', 'VICTORIA', 'CABORCA'] | TEST:54 
Processing: SIN |['MONTEMORELOS', 'MONTERREY', 'NUEVO LAREDO', 'PIEDRAS NEGRAS', 'REYNOSA', 'SABINAS', 'SALTILLO', 'TAMPICO', 'VICTORIA', 'CABORCA'] | TEST:55 
Processing: SIN |['MONTEMORELOS', 'MONTERREY', 'NUEVO LAREDO', 'PIEDRAS NEGRAS', 'REYNOSA', 'SABINAS', 'SALTILLO', 'TAMPICO', 'VICTORIA', 'CABORCA'] | TEST:56 
Processing: SIN |['

Processing: SIN |['MONTEMORELOS', 'MONTERREY', 'NUEVO LAREDO', 'PIEDRAS NEGRAS', 'REYNOSA', 'SABINAS', 'SALTILLO', 'TAMPICO', 'VICTORIA', 'CABORCA'] | TEST:103 
Processing: SIN |['MONTEMORELOS', 'MONTERREY', 'NUEVO LAREDO', 'PIEDRAS NEGRAS', 'REYNOSA', 'SABINAS', 'SALTILLO', 'TAMPICO', 'VICTORIA', 'CABORCA'] | TEST:104 
Processing: SIN |['MONTEMORELOS', 'MONTERREY', 'NUEVO LAREDO', 'PIEDRAS NEGRAS', 'REYNOSA', 'SABINAS', 'SALTILLO', 'TAMPICO', 'VICTORIA', 'CABORCA'] | TEST:105 
Processing: SIN |['MONTEMORELOS', 'MONTERREY', 'NUEVO LAREDO', 'PIEDRAS NEGRAS', 'REYNOSA', 'SABINAS', 'SALTILLO', 'TAMPICO', 'VICTORIA', 'CABORCA'] | TEST:106 
Processing: SIN |['CULIACAN', 'GUASAVE', 'GUAYMAS', 'HERMOSILLO', 'LOS MOCHIS', 'MAZATLAN', 'NAVOJOA', 'NOGALES', 'OBREGON', 'CAMARGO'] | TEST:107 
Processing: SIN |['CULIACAN', 'GUASAVE', 'GUAYMAS', 'HERMOSILLO', 'LOS MOCHIS', 'MAZATLAN', 'NAVOJOA', 'NOGALES', 'OBREGON', 'CAMARGO'] | TEST:108 
Processing: SIN |['CULIACAN', 'GUASAVE', 'GUAYMAS', 'HERMOSI

Processing: SIN |['CULIACAN', 'GUASAVE', 'GUAYMAS', 'HERMOSILLO', 'LOS MOCHIS', 'MAZATLAN', 'NAVOJOA', 'NOGALES', 'OBREGON', 'CAMARGO'] | TEST:159 
Processing: SIN |['CASAS GRANDES', 'CHIHUAHUA', 'CUAUHTEMOC', 'DURANGO', 'JUAREZ', 'LAGUNA', 'AGUASCALIENTES', 'APATZINGAN', 'CELAYA', 'CIENEGA'] | TEST:160 
Processing: SIN |['CASAS GRANDES', 'CHIHUAHUA', 'CUAUHTEMOC', 'DURANGO', 'JUAREZ', 'LAGUNA', 'AGUASCALIENTES', 'APATZINGAN', 'CELAYA', 'CIENEGA'] | TEST:161 
Processing: SIN |['CASAS GRANDES', 'CHIHUAHUA', 'CUAUHTEMOC', 'DURANGO', 'JUAREZ', 'LAGUNA', 'AGUASCALIENTES', 'APATZINGAN', 'CELAYA', 'CIENEGA'] | TEST:162 
Processing: SIN |['CASAS GRANDES', 'CHIHUAHUA', 'CUAUHTEMOC', 'DURANGO', 'JUAREZ', 'LAGUNA', 'AGUASCALIENTES', 'APATZINGAN', 'CELAYA', 'CIENEGA'] | TEST:163 
Processing: SIN |['CASAS GRANDES', 'CHIHUAHUA', 'CUAUHTEMOC', 'DURANGO', 'JUAREZ', 'LAGUNA', 'AGUASCALIENTES', 'APATZINGAN', 'CELAYA', 'CIENEGA'] | TEST:164 
Processing: SIN |['CASAS GRANDES', 'CHIHUAHUA', 'CUAUHTEMOC', 

Processing: SIN |['CASAS GRANDES', 'CHIHUAHUA', 'CUAUHTEMOC', 'DURANGO', 'JUAREZ', 'LAGUNA', 'AGUASCALIENTES', 'APATZINGAN', 'CELAYA', 'CIENEGA'] | TEST:211 
Processing: SIN |['CASAS GRANDES', 'CHIHUAHUA', 'CUAUHTEMOC', 'DURANGO', 'JUAREZ', 'LAGUNA', 'AGUASCALIENTES', 'APATZINGAN', 'CELAYA', 'CIENEGA'] | TEST:212 
Processing: SIN |['COLIMA', 'FRESNILLO', 'GUADALAJARA', 'IRAPUATO', 'IXMIQUILPAN', 'JIQUILPAN', 'LEON', 'LOS ALTOS', 'MANZANILLO', 'MATEHUALA'] | TEST:213 
Processing: SIN |['COLIMA', 'FRESNILLO', 'GUADALAJARA', 'IRAPUATO', 'IXMIQUILPAN', 'JIQUILPAN', 'LEON', 'LOS ALTOS', 'MANZANILLO', 'MATEHUALA'] | TEST:214 
Processing: SIN |['COLIMA', 'FRESNILLO', 'GUADALAJARA', 'IRAPUATO', 'IXMIQUILPAN', 'JIQUILPAN', 'LEON', 'LOS ALTOS', 'MANZANILLO', 'MATEHUALA'] | TEST:215 
Processing: SIN |['COLIMA', 'FRESNILLO', 'GUADALAJARA', 'IRAPUATO', 'IXMIQUILPAN', 'JIQUILPAN', 'LEON', 'LOS ALTOS', 'MANZANILLO', 'MATEHUALA'] | TEST:216 
Processing: SIN |['COLIMA', 'FRESNILLO', 'GUADALAJARA', 'IRA

...

Processing: SIN |['MINAS', 'MORELIA', 'QUERETARO', 'SALVATIERRA', 'SAN JUAN DEL RIO', 'SAN LUIS POTOSI', 'TEPIC VALLARTA', 'URUAPAN', 'ZACAPU', 'ZACATECAS'] | TEST:313 
Processing: SIN |['MINAS', 'MORELIA', 'QUERETARO', 'SALVATIERRA', 'SAN JUAN DEL RIO', 'SAN LUIS POTOSI', 'TEPIC VALLARTA', 'URUAPAN', 'ZACAPU', 'ZACATECAS'] | TEST:314 
Processing: SIN |['MINAS', 'MORELIA', 'QUERETARO', 'SALVATIERRA', 'SAN JUAN DEL RIO', 'SAN LUIS POTOSI', 'TEPIC VALLARTA', 'URUAPAN', 'ZACAPU', 'ZACATECAS'] | TEST:315 
Processing: SIN |['MINAS', 'MORELIA', 'QUERETARO', 'SALVATIERRA', 'SAN JUAN DEL RIO', 'SAN LUIS POTOSI', 'TEPIC VALLARTA', 'URUAPAN', 'ZACAPU', 'ZACATECAS'] | TEST:316 
Processing: SIN |['MINAS', 'MORELIA', 'QUERETARO', 'SALVATIERRA', 'SAN JUAN DEL RIO', 'SAN LUIS POTOSI', 'TEPIC VALLARTA', 'URUAPAN', 'ZACAPU', 'ZACATECAS'] | TEST:317 
Processing: SIN |['MINAS', 'MORELIA', 'QUERETARO', 'SALVATIERRA', 'SAN JUAN DEL RIO', 'SAN LUIS POTOSI', 'TEPIC VALLARTA', 'URUAPAN', 'ZACAPU', 'ZACATECAS'

Processing: SIN |['ZAMORA', 'ZAPOTLAN', 'ACAPULCO', 'CHILPANCINGO', 'CHONTALPA', 'COATZACOALCOS', 'CORDOBA', 'CUAUTLA', 'CUERNAVACA', 'HUAJUAPAN'] | TEST:365 
Processing: SIN |['ZAMORA', 'ZAPOTLAN', 'ACAPULCO', 'CHILPANCINGO', 'CHONTALPA', 'COATZACOALCOS', 'CORDOBA', 'CUAUTLA', 'CUERNAVACA', 'HUAJUAPAN'] | TEST:366 
Processing: SIN |['ZAMORA', 'ZAPOTLAN', 'ACAPULCO', 'CHILPANCINGO', 'CHONTALPA', 'COATZACOALCOS', 'CORDOBA', 'CUAUTLA', 'CUERNAVACA', 'HUAJUAPAN'] | TEST:367 
Processing: SIN |['ZAMORA', 'ZAPOTLAN', 'ACAPULCO', 'CHILPANCINGO', 'CHONTALPA', 'COATZACOALCOS', 'CORDOBA', 'CUAUTLA', 'CUERNAVACA', 'HUAJUAPAN'] | TEST:368 
Processing: SIN |['ZAMORA', 'ZAPOTLAN', 'ACAPULCO', 'CHILPANCINGO', 'CHONTALPA', 'COATZACOALCOS', 'CORDOBA', 'CUAUTLA', 'CUERNAVACA', 'HUAJUAPAN'] | TEST:369 
Processing: SIN |['ZAMORA', 'ZAPOTLAN', 'ACAPULCO', 'CHILPANCINGO', 'CHONTALPA', 'COATZACOALCOS', 'CORDOBA', 'CUAUTLA', 'CUERNAVACA', 'HUAJUAPAN'] | TEST:370 
Processing: SIN |['ZAMORA', 'ZAPOTLAN', 'ACAPU

Processing: SIN |['HUATULCO', 'IGUALA', 'IZUCAR', 'LOS RIOS', 'LOS TUXTLAS', 'MORELOS', 'OAXACA', 'ORIZABA', 'POZA RICA', 'PUEBLA'] | TEST:422 
Processing: SIN |['HUATULCO', 'IGUALA', 'IZUCAR', 'LOS RIOS', 'LOS TUXTLAS', 'MORELOS', 'OAXACA', 'ORIZABA', 'POZA RICA', 'PUEBLA'] | TEST:423 
Processing: SIN |['HUATULCO', 'IGUALA', 'IZUCAR', 'LOS RIOS', 'LOS TUXTLAS', 'MORELOS', 'OAXACA', 'ORIZABA', 'POZA RICA', 'PUEBLA'] | TEST:424 
Processing: SIN |['SAN CRISTOBAL', 'SAN MARTIN', 'TAPACHULA', 'TECAMACHALCO', 'TEHUACAN', 'TEHUANTEPEC', 'TEZIUTLAN', 'TLAXCALA', 'TUXTLA', 'VERACRUZ'] | TEST:425 
Processing: SIN |['SAN CRISTOBAL', 'SAN MARTIN', 'TAPACHULA', 'TECAMACHALCO', 'TEHUACAN', 'TEHUANTEPEC', 'TEZIUTLAN', 'TLAXCALA', 'TUXTLA', 'VERACRUZ'] | TEST:426 
Processing: SIN |['SAN CRISTOBAL', 'SAN MARTIN', 'TAPACHULA', 'TECAMACHALCO', 'TEHUACAN', 'TEHUANTEPEC', 'TEZIUTLAN', 'TLAXCALA', 'TUXTLA', 'VERACRUZ'] | TEST:427 
Processing: SIN |['SAN CRISTOBAL', 'SAN MARTIN', 'TAPACHULA', 'TECAMACHALCO'

Processing: SIN |['SAN CRISTOBAL', 'SAN MARTIN', 'TAPACHULA', 'TECAMACHALCO', 'TEHUACAN', 'TEHUANTEPEC', 'TEZIUTLAN', 'TLAXCALA', 'TUXTLA', 'VERACRUZ'] | TEST:473 
Processing: SIN |['SAN CRISTOBAL', 'SAN MARTIN', 'TAPACHULA', 'TECAMACHALCO', 'TEHUACAN', 'TEHUANTEPEC', 'TEZIUTLAN', 'TLAXCALA', 'TUXTLA', 'VERACRUZ'] | TEST:474 
Processing: SIN |['SAN CRISTOBAL', 'SAN MARTIN', 'TAPACHULA', 'TECAMACHALCO', 'TEHUACAN', 'TEHUANTEPEC', 'TEZIUTLAN', 'TLAXCALA', 'TUXTLA', 'VERACRUZ'] | TEST:475 
Processing: SIN |['SAN CRISTOBAL', 'SAN MARTIN', 'TAPACHULA', 'TECAMACHALCO', 'TEHUACAN', 'TEHUANTEPEC', 'TEZIUTLAN', 'TLAXCALA', 'TUXTLA', 'VERACRUZ'] | TEST:476 
Processing: SIN |['SAN CRISTOBAL', 'SAN MARTIN', 'TAPACHULA', 'TECAMACHALCO', 'TEHUACAN', 'TEHUANTEPEC', 'TEZIUTLAN', 'TLAXCALA', 'TUXTLA', 'VERACRUZ'] | TEST:477 
Processing: SIN |['VILLAHERMOSA', 'XALAPA', 'ZIHUATANEJO', 'CAMPECHE', 'CANCUN', 'CARMEN', 'CHETUMAL', 'MERIDA', 'MOTUL TIZIMIN', 'RIVIERA MAYA'] | TEST:478 
Processing: SIN |['VIL

Processing: SIN |['VILLAHERMOSA', 'XALAPA', 'ZIHUATANEJO', 'CAMPECHE', 'CANCUN', 'CARMEN', 'CHETUMAL', 'MERIDA', 'MOTUL TIZIMIN', 'RIVIERA MAYA'] | TEST:525 
Processing: SIN |['VILLAHERMOSA', 'XALAPA', 'ZIHUATANEJO', 'CAMPECHE', 'CANCUN', 'CARMEN', 'CHETUMAL', 'MERIDA', 'MOTUL TIZIMIN', 'RIVIERA MAYA'] | TEST:526 
Processing: SIN |['VILLAHERMOSA', 'XALAPA', 'ZIHUATANEJO', 'CAMPECHE', 'CANCUN', 'CARMEN', 'CHETUMAL', 'MERIDA', 'MOTUL TIZIMIN', 'RIVIERA MAYA'] | TEST:527 
Processing: SIN |['VILLAHERMOSA', 'XALAPA', 'ZIHUATANEJO', 'CAMPECHE', 'CANCUN', 'CARMEN', 'CHETUMAL', 'MERIDA', 'MOTUL TIZIMIN', 'RIVIERA MAYA'] | TEST:528 
Processing: SIN |['VILLAHERMOSA', 'XALAPA', 'ZIHUATANEJO', 'CAMPECHE', 'CANCUN', 'CARMEN', 'CHETUMAL', 'MERIDA', 'MOTUL TIZIMIN', 'RIVIERA MAYA'] | TEST:529 
Processing: SIN |['VILLAHERMOSA', 'XALAPA', 'ZIHUATANEJO', 'CAMPECHE', 'CANCUN', 'CARMEN', 'CHETUMAL', 'MERIDA', 'MOTUL TIZIMIN', 'RIVIERA MAYA'] | TEST:530 
Processing: SIN |['TICUL'] | TEST:531 
Processing: S

In [15]:
#Dataframe for MTR prices

MT_DF_MTR = pd.DataFrame({
    "SISTEMA":Sistemas_MTR,
    "ZONA":Zonas_MTR,
    "FECHA":Fechas_MTR,
    "HORA":Horas_MTR,
    "PRECIO MTR":pz_MTR,
    "PRECIO ENERGIA MTR":pz_ene_MTR,
    "PRECIO PERDIDA MTR":pz_per_MTR,
    "PRECIO CONGESTION MTR":pz_cng_MTR,
})
MT_DF_MTR= MT_DF_MTR.sort_values(["SISTEMA","ZONA","FECHA"], ascending=[True, True,True])
MT_DF_MTR

,SISTEMA,ZONA,FECHA,HORA,PRECIO MTR,PRECIO ENERGIA MTR,PRECIO PERDIDA MTR,PRECIO CONGESTION MTR
0,BCA,ENSENADA,2018-01-01,1,525.48,508.92,16.56,0
1,BCA,ENSENADA,2018-01-01,2,520.96,505.26,15.7,0
2,BCA,ENSENADA,2018-01-01,3,517.76,502.7,15.06,0
3,BCA,ENSENADA,2018-01-01,4,512.98,498.1,14.88,0
4,BCA,ENSENADA,2018-01-01,5,516.38,501.2,15.18,0
5,BCA,ENSENADA,2018-01-01,6,505.71,490.85,14.86,0
6,BCA,ENSENADA,2018-01-01,7,500.34,485.57,14.76,0
7,BCA,ENSENADA,2018-01-01,8,493.47,479.62,13.85,0
8,BCA,ENSENADA,2018-01-01,9,499.12,485.42,13.69,0
9,BCA,ENSENADA,2018-01-01,10,528.71,513.92,14.79,0


In [16]:
MT_DF_MTR.to_csv("Master_Table_MTR.csv")